In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import IPython.display as display

import os
import re
import base64
import json
from string import Template

import warnings
warnings.filterwarnings('ignore')

/mnt/c/Users/denad/OneDrive/Documents/GitHub/uefa-championship-analysis-1/.venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


### Data Loading

In [2]:
#============== Concat all matches over the years into one file ===============#
# Path to the directory containing the yearly matches CSV files
data_dir = '../inputs/matches/euro/'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

all_matches = pd.DataFrame()

for file in csv_files:
    # Extract the year from the filename (format 'YYYY.csv')
    year = int(file.split('.')[0])
    df = pd.read_csv(os.path.join(data_dir, file))
    
    # Add a new column 'year' to the DataFrame
    df['year'] = year
    
    # Append the DataFrame to the combined DataFrame
    all_matches = pd.concat([all_matches, df], ignore_index=True)

all_matches.head()

,id_match,home_team,away_team,home_team_code,away_team_code,home_score,away_score,home_penalty,away_penalty,home_score_total,...,penalties_missed,penalties,red_cards,game_referees,stadium_city,stadium_name,stadium_name_media,stadium_name_official,stadium_name_event,stadium_name_sponsor
0,4025,USSR,Yugoslavia,URS,YUG,1.0,1.0,NaN,NaN,2.0,...,NaN,NaN,NaN,[],Paris,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes
1,4024,Czechoslovakia,France,TCH,FRA,2.0,0.0,NaN,NaN,2.0,...,NaN,NaN,NaN,[],Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Orange Vélodrome
2,4023,Czechoslovakia,USSR,TCH,URS,0.0,3.0,NaN,NaN,0.0,...,NaN,NaN,NaN,[],Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Orange Vélodrome
3,4022,France,Yugoslavia,FRA,YUG,4.0,5.0,NaN,NaN,4.0,...,NaN,NaN,NaN,[],Paris,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes
4,3996,Spain,USSR,ESP,URS,2.0,1.0,NaN,NaN,2.0,...,NaN,NaN,NaN,[],Madrid,Estadio Santiago Bernabéu,Estadio Santiago Bernabéu,Estadio Santiago Bernabéu,Estadio Santiago Bernabéu,NaN


In [4]:
all_matches.shape

(388, 47)

In [ ]:
all_matches.info()

### Data Preprocessing

In [3]:
#====== drop all matches that have not taken place yet (referring to 2024) =====#
all_matches = all_matches.loc[all_matches['status'] == 'FINISHED']

all_matches.to_csv('../data/all_matches.csv', index=False)
all_matches.shape

(350, 47)

### Participants

#### Note on Historical Countries in UEFA EURO Data (1960 - 2024)

The data spans from 1960 to 2024 and includes some countries that no longer exist (Find out more [here](https://en.wikipedia.org/wiki/UEFA)). Below is the information about these historical countries and the modern countries they have become:

1. **CIS (Commonwealth of Independent States)**
   - **Existed:** 1992
   - **Countries:** Armenia 🇦🇲, Azerbaijan 🇦🇿, Belarus 🇧🇾, Georgia 🇬🇪, Kazakhstan 🇰🇿, Kyrgyzstan 🇰🇬, Moldova 🇲🇩, Russia 🇷🇺, Tajikistan 🇹🇯, Turkmenistan 🇹🇲, Ukraine 🇺🇦, Uzbekistan 🇺🇿

2. **USSR (Union of Soviet Socialist Republics)**
   - **Existed:** 1922–1991
   - **Countries:** Armenia 🇦🇲, Azerbaijan 🇦🇿, Belarus 🇧🇾, Estonia 🇪🇪, Georgia 🇬🇪, Kazakhstan 🇰🇿, Kyrgyzstan 🇰🇬, Latvia 🇱🇻, Lithuania 🇱🇹, Moldova 🇲🇩, Russia 🇷🇺, Tajikistan 🇹🇯, Turkmenistan 🇹🇲, Ukraine 🇺🇦, Uzbekistan 🇺🇿

3. **West Germany (Federal Republic of Germany)**
   - **Existed:** 1949–1990
   - **Countries:** Part of the current unified Germany 🇩🇪

4. **Yugoslavia**
   - **Existed:** 1918–1992 (as the Kingdom of Yugoslavia and later the Socialist Federal Republic of Yugoslavia)
   - **Countries:** Bosnia and Herzegovina 🇧🇦, Croatia 🇭🇷, Macedonia 🇲🇰, Montenegro 🇲🇪, Serbia 🇷🇸, Slovenia 🇸🇮

5. **Czechoslovakia**
   - **Existed:** 1918–1992
   - **Countries:** Czech Republic 🇨🇿, Slovakia 🇸🇰

### Goal Trend Analysis over years


In [ ]:
one_match = all_matches.loc[all_matches['id_match'] == 2024491]
# one_match['home_team']
one_match['red_cards'].iloc[0]

In [ ]:
all_matches.loc[all_matches['red_cards'].notnull()]['red_cards'].iloc[0]

In [6]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 387
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_match               350 non-null    int64  
 1   home_team              350 non-null    object 
 2   away_team              350 non-null    object 
 3   home_team_code         350 non-null    object 
 4   away_team_code         350 non-null    object 
 5   home_score             350 non-null    float64
 6   away_score             350 non-null    float64
 7   home_penalty           22 non-null     float64
 8   away_penalty           22 non-null     float64
 9   home_score_total       350 non-null    float64
 10  away_score_total       350 non-null    float64
 11  winner                 285 non-null    object 
 12  winner_reason          350 non-null    object 
 13  year                   350 non-null    int64  
 14  date                   350 non-null    object 
 15  date_time  

### Extract Tournament Winners Data

In [4]:
# Extract final match details for each year
finals = all_matches[all_matches['round'] == 'FINAL']

tournament_winners_list = []
for year, group in finals.groupby('year'):
    winner = group['winner'].values[0]
    runner_up = group['away_team'].values[0] if group['winner'].values[0] == group['home_team'].values[0] else group['home_team'].values[0]
    score = f"{int(group['home_score'].values[0])}-{int(group['away_score'].values[0])}"
    participants = pd.concat([all_matches[all_matches['year'] == year]['home_team'], 
                              all_matches[all_matches['year'] == year]['away_team']]).unique()
    total_participants = len(participants)
    
    tournament_winners_list.append({
        'year': year,
        'totalParticipants': total_participants,
        'winner': winner,
        'runnerUp': runner_up,
        'score': score
    })

tournament_winners = pd.DataFrame(tournament_winners_list)
tournament_winners.to_csv('../data/tournament_winners.csv', index=False)

### Extract First-time Participations for each country

In [5]:
# Extract first participation year for each country
home_teams = all_matches[['year', 'home_team', 'home_team_code']].drop_duplicates()
away_teams = all_matches[['year', 'away_team', 'away_team_code']].drop_duplicates()

teams = pd.concat([home_teams.rename(columns={'home_team': 'team', 'home_team_code': 'team_code'}),
                   away_teams.rename(columns={'away_team': 'team', 'away_team_code': 'team_code'})])

first_participation = teams.groupby('team').year.min().reset_index()
first_participation = first_participation.sort_values('year')
first_participation = first_participation.merge(teams[['team', 'team_code']].drop_duplicates(), on='team')

# Calculate total participations, total wins, and final appearances
total_participations = teams.groupby('team').size().reset_index(name='total_participations')
total_wins = finals['winner'].value_counts().reset_index(name='total_wins').rename(columns={'winner': 'team'})
final_appearances = pd.concat([finals['home_team'], finals['away_team']]).value_counts().reset_index(name='final_appearances').rename(columns={'index': 'team'})

# Merge the new fields with the first participation data
first_participation = first_participation.merge(total_participations, on='team', how='left')
first_participation = first_participation.merge(total_wins, on='team', how='left')
first_participation = first_participation.merge(final_appearances, on='team', how='left')

# Fill NaN values with 0 for total_wins and final_appearances
first_participation['total_wins'] = first_participation['total_wins'].fillna(0).astype(int)
first_participation['final_appearances'] = first_participation['final_appearances'].fillna(0).astype(int)

first_participation.to_csv('../data/first_participation.csv', index=False)
first_participation.head()

,team,year,team_code,total_participations,total_wins,final_appearances
0,Czechoslovakia,1960,TCH,4,1,1
1,France,1960,FRA,20,2,3
2,Yugoslavia,1960,YUG,8,0,3
3,USSR,1960,URS,7,1,4
4,Denmark,1964,DEN,19,1,1


In [7]:
# Encode flag images to embed them in the HTML file
def encode_images(directory):
    encoded_images = {}
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            with open(os.path.join(directory, filename), "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                team_code = filename.split('.')[0]
                encoded_images[team_code] = encoded_string
    return encoded_images

flags_directory = '../inputs/logos/'
encoded_flags = encode_images(flags_directory)

In [8]:
viz1_html = Template(r"""
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>UEFA Euro Timeline</title>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Montserrat', sans-serif;
            background-color: #161616;
            color: #ffffff;
        }

        #timeline-container {
            width: 100%;
            height: 600px;
            background: linear-gradient(90deg, #1A365D, #24497c);
            border-radius: 5px;
            box-shadow: 0px 0px 4px 0px black;
        }

        .title {
            font-weight: 700;
            font-size: 28px;
            fill: #FFD700;
            /* UEFA gold color */
        }

        .subtitle {
            font-weight: 400;
            font-size: 18px;
            fill: white;
        }

        .description {
            font-weight: 400;
            font-size: 13px;
            fill: white;
        }

        .hint {
            font-size: 12px;
            fill: #779dd2;
        }

        .timeline-line {
            stroke: #FFD700;
            stroke-width: 2;
        }

        .country-flag {
            cursor: pointer;
        }

        .country-label {
            font-size: 12px;
            fill: #FFFFFF;
        }

        .year-label {
            font-size: 14px;
            fill: #FFD700;
            font-weight: bold;
        }

        .country-item {
            pointer-events: all;
        }

        .year-hover-area {
            pointer-events: all;
        }

        .country-hover-area {
            pointer-events: all;
        }

        .tooltip {
            font-family: 'Montserrat', sans-serif;
            position: absolute;
            background-color: rgba(255, 255, 255, 0.9);
            color: #333;
            border-radius: 5px;
            padding: 10px;
            font-size: 12px;
            pointer-events: none;
        }

        .country-tooltip {
            z-index: 1001;
        }

        .year-tooltip {
            z-index: 1000;
        }

        button {
            font-family: 'Montserrat', sans-serif;
            background-color: #FFD700;
            color: #1A365D;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s;
        }

        button:hover {
            background-color: #FFC000;
        }
    </style>
</head>

<body>
    <div id="timeline-container"></div>
    <div id="start-button" style="position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%);"></div>
    <div id="replay-button" style="position: absolute; bottom: 20px; left: 20px;"></div>

    <script type="module">
        'use strict';
        import * as d3 from 'https://cdn.skypack.dev/d3';
        (async () => {

            const participationData = d3.csvParse(`$df1`, d3.autoType)
            const winnersData = d3.csvParse(`$df2`, d3.autoType)
            const flagImages = $flag_images

            const tournamentWinners = {};
            winnersData.forEach(d => {
                tournamentWinners[d.year] = {
                    winner: d.winner,
                    runnerUp: d.runnerUp,
                    score: d.score,
                    totalParticipants: +d.totalParticipants
                };
            });

            createTimeline(participationData, tournamentWinners);

            function createTimeline(data, tournamentWinners) {
                const width = document.getElementById('timeline-container').offsetWidth;
                const height = 600;
                const margin = { top: 150, right: 50, bottom: 150, left: 50 };
                const fontSize = 14;
                const lineHeight = 1.5;
                const lineHeightPx = 20;
                const singleTickHeight = 40;

                // Add expansion years data
                const expansionYears = [
                    { year: 1980, text: "Expansion to 8 teams" },
                    { year: 1996, text: "Expansion to 16 teams" },
                    { year: 2016, text: "Expansion to 24 teams" }
                ];

                // Group data by year
                const groupedData = d3.group(data, d => d.year);
                const preparedData = Array.from(groupedData, ([year, values]) => ({
                    year: +year,
                    teams: values,
                    numberOfLines: values.length
                }));

                let activeTooltip = null;
                let animationStarted = false;

                d3.select("#timeline-container").selectAll("*").remove();

                const svg = d3.select("#timeline-container")
                    .append("svg")
                    .attr("width", width)
                    .attr("height", height);    // Add title
                svg.append("text")
                    .attr("class", "title")
                    .attr("x", margin.left)
                    .attr("y", margin.top - 100)
                    .text("UEFA EURO Championship Timeline");

                // Add subtitle
                svg.append("text")
                    .attr("class", "subtitle")
                    .attr("x", margin.left)
                    .attr("y", margin.top - 70)
                    .text("The Evolution of European Football");

                // Add description
                const description = svg.append("g")
                    .attr("transform", `translate(${margin.left}, ${margin.top - 40})`);

                description.append("text")
                    .attr("class", "description")
                    .attr("y", 0)
                    .text("This timeline showcases the first appearances of countries");

                description.append("text")
                    .attr("class", "description")
                    .attr("y", 20)
                    .text("in the UEFA EURO Championship from 1960 to 2024.");

                // Add hint
                const hint = svg.append("g")
                    .attr("transform", `translate(${width - margin.right}, ${margin.top - 100})`)
                    .attr("text-anchor", "end");

                hint.append("text")
                    .attr("class", "hint")
                    .attr("y", 0)
                    .text("🖱️ Hover/click for more info");

                hint.append("text")
                    .attr("class", "hint")
                    .attr("y", 20)
                    .text("on years and countries");

                // Create start button
                const startButton = d3.select("#start-button")
                    .append("button")
                    .text("Start")
                    .style("font-size", "20px")
                    .style("padding", "10px 20px")
                    .on("click", function () {
                        d3.select(this).style("display", "none");
                        animationStarted = true;
                        animateLine();
                    });

                // Create replay button (initially hidden)
                const replayButton = d3.select("#replay-button")
                    .append("button")
                    .text("Replay")
                    .style("font-size", "16px")
                    .style("padding", "5px 10px")
                    .style("display", "none")
                    .on("click", function () {
                        d3.select(this).style("display", "none");
                        resetAndAnimate();
                    });

                const x = d3.scaleLinear()
                    .domain(d3.extent(preparedData, d => d.year))
                    .range([margin.left, width - margin.right]);

                const line = svg.append("line")
                    .attr("x1", margin.left)
                    .attr("y1", height / 2)
                    .attr("x2", margin.left)
                    .attr("y2", height / 2)
                    .attr("stroke", "#FFD700")
                    .attr("stroke-width", 2)
                    ;

                // Add expansion year lines (initially hidden)
                const expansionLines = svg.selectAll(".expansion-line")
                    .data(expansionYears)
                    .enter()
                    .append("g")
                    .attr("class", "expansion-line")
                    .attr("transform", d => `translate(${x(d.year)}, 0)`)
                    .style("opacity", 0);

                expansionLines.append("line")
                    .attr("y1", height / 2 - height * 0.25)
                    .attr("y2", height / 2 + height * 0.3)
                    .attr("stroke", "#779dd2")
                    .attr("stroke-width", 3)
                    .attr("stroke-dasharray", "5,5");

                expansionLines.append("text")
                    .attr("y", height / 2 + height * 0.3 + 20)
                    .text(d => d.text)
                    .attr("text-anchor", "middle")
                    .attr("font-size", fontSize)
                    .attr("font-weight", "bold")
                    .attr("fill", "#779dd2");

                const events = svg.selectAll(".event")
                    .data(preparedData)
                    .enter()
                    .append("g")
                    .attr("class", "event")
                    .attr("transform", d => `translate(${x(d.year)}, ${height / 2})`)
                    .style("opacity", 0);


                events.append("line")
                    .attr("class", "country-tick")
                    .attr("y1", 0)
                    .attr("y2", 0)
                    .attr("stroke", "#FFD700")
                    .attr("stroke-width", 2);

                events.append("circle")
                    .attr("r", 3)
                    .attr("fill", "white")
                    .attr("stroke", "black");

                events.append("text")
                    .attr("class", "year-text")
                    .attr("y", (d, i) => i % 2 === 0 ? 7 : -7)
                    .text(d => d.year)
                    .attr("text-anchor", "middle")
                    .attr("dominant-baseline", (d, i) => i % 2 === 0 ? "hanging" : "alphabetic")
                    .attr("font-size", fontSize)
                    .style("opacity", 0)
                    .style("paint-order", "stroke")
                    .style("stroke", "white")
                    .style("stroke-width", "3px")
                    .style("fill", "black");

                events.on("mouseover", function (event, d) {
                    if (animationStarted) {
                        showYearTooltip(d, event.pageX, event.pageY);
                    }

                }).on("mouseout", function () {
                    if ((!activeTooltip || !activeTooltip.classed('country-tooltip')) && animationStarted) {
                        hideTooltip();
                    }
                });

                const countryGroups = events.append("g")
                    .attr("class", "country-group")
                    .attr("transform", (d, i) => {
                        const yOffset = i % 2 === 0 ?
                            -(singleTickHeight + 2 + (d.teams.length - 1) * lineHeightPx) :
                            singleTickHeight + 2;
                        return `translate(0, ${yOffset})`;
                    });

                countryGroups.selectAll(".country-item")
                    .data(d => d.teams)
                    .enter()
                    .append("g")
                    .attr("class", "country-item")
                    .attr("transform", (d, i, nodes) => {
                        const parentData = d3.select(nodes[i].parentNode).datum();
                        const yOffset = parentData.year % 4 === 0 ?
                            i * lineHeightPx :
                            -i * lineHeightPx;
                        return `translate(0, ${yOffset})`;
                    });

                countryGroups.selectAll(".country-item")
                    .append("image")
                    .attr("xlink:href", d => `data:image/png;base64,${flagImages[d.team_code]}`)
                    .attr("width", 18)
                    .attr("height", 18)
                    .attr("x", -32)
                    .attr("y", -12)
                    .style("opacity", 0);

                countryGroups.selectAll(".country-item")
                    .append("text")
                    .attr("x", 0)
                    .attr("y", 0)
                    .text(d => d.team_code)
                    .attr("text-anchor", "middle")
                    .attr("dominant-baseline", "middle")
                    .attr("font-size", fontSize)
                    .style("opacity", 0)
                    .style("paint-order", "stroke")
                    .style("stroke", "white")
                    .style("stroke-width", "1px")
                    .style("fill", "white");

                countryGroups.selectAll(".country-item")
                    .on("mouseover", function (event, d) {
                        if (animationStarted) {
                            event.stopPropagation();
                            showCountryTooltip(d, event.pageX, event.pageY);
                        }

                    })
                    .on("mouseout", function (event) {
                        // Only hide the tooltip if we're not entering another country item
                        if ((!event.relatedTarget || !d3.select(event.relatedTarget).classed('country-item')) && animationStarted) {
                            hideTooltip();
                        }
                    });

                function showYearTooltip(data, x, y) {
                    if (activeTooltip && activeTooltip.classed('country-tooltip')) return;

                    hideTooltip();
                    const winner = tournamentWinners[data.year];
                    const tooltip = d3.select('body').append('div')
                        .attr('class', 'tooltip year-tooltip')
                        .style('left', `${x + 10}px`)
                        .style('top', `${y - 10}px`)
                        .style('z-index', 1000);

                    tooltip.html(`
                    <strong style="font-size: 14px;">${data.year}</strong><br>
                    <hr>
                    <span style="font-weight: bold;">Winner:</span> ${winner.winner}<br>
                    <span style="font-weight: bold;">Runner-up:</span> ${winner.runnerUp}<br>
                    <span style="font-weight: bold;">Score:</span> ${winner.score}<br>
                    <span style="font-weight: bold;">Participants:</span> ${winner.totalParticipants}
                `);

                    activeTooltip = tooltip;
                }


                function showCountryTooltip(data, x, y) {
                    hideTooltip();
                    const tooltip = d3.select('body').append('div')
                        .attr('class', 'tooltip country-tooltip')
                        .style('left', `${x + 10}px`)
                        .style('top', `${y - 10}px`)
                        .style('z-index', 1001);

                    tooltip.html(`
                    <strong style="font-size: 14px;">${data.team}</strong><br>
                    <hr>
                    <span style="font-weight: bold;">Total Participations:</span> ${data.total_participations}<br>
                    <span style="font-weight: bold;">Wins:</span> ${data.total_wins}<br>
                    <span style="font-weight: bold;">Final Appearances:</span> ${data.final_appearances}
                `);

                    activeTooltip = tooltip;
                }

                function hideTooltip() {
                    if (activeTooltip) {
                        activeTooltip.remove();
                        activeTooltip = null;
                    }
                }

                function resetAndAnimate() {
                    animationStarted = false;
                    // Reset the timeline
                    line.attr("x2", margin.left);
                    events.style("opacity", 0);
                    events.select(".country-tick")
                        .attr("y1", 0)
                        .attr("y2", 0);
                    events.select(".year-text").style("opacity", 0);
                    countryGroups.selectAll(".country-item image").style("opacity", 0);
                    countryGroups.selectAll(".country-item text").style("opacity", 0);
                    expansionLines.style("opacity", 0);

                    // Start the animation
                    animateLine();
                }

                function animateLine() {
                    animationStarted = true;
                    const events = d3.selectAll('.event').nodes();
                    let currentIndex = 0;

                    function animateNextEvent() {
                        // Check if animation is complete
                        if (currentIndex >= events.length) {
                            // Show replay button
                            replayButton.style("display", "block");
                            return;
                        }

                        const event = d3.select(events[currentIndex]);
                        const eventData = event.datum();
                        const eventX = x(eventData.year);
                        const nextEventX = currentIndex < events.length - 1 ? x(d3.select(events[currentIndex + 1]).datum().year) : width - margin.right;

                        const totalAnimationDuration = 800; // Adjust this value to control the overall speed

                        // Grow main line to the next event
                        line.transition()
                            .duration(totalAnimationDuration)
                            .ease(d3.easeLinear)
                            .attr("x2", nextEventX)
                            .on("end", () => {
                                currentIndex++;
                                animateNextEvent();
                            });

                        event.style("opacity", 1);

                        // Animate tick
                        const tick = event.select(".country-tick");
                        tick.transition()
                            .duration(500)
                            .attr("y1", (d, i) => currentIndex % 2 === 0 ? -singleTickHeight + 4 : 0)
                            .attr("y2", (d, i) => currentIndex % 2 === 0 ? 0 : singleTickHeight - 8);

                        // Animate year text
                        event.select(".year-text")
                            .transition()
                            .duration(500)
                            .style("opacity", 1);

                        // Animate country codes and flags
                        const countryItems = event.select(".country-group").selectAll(".country-item");
                        const totalItems = countryItems.size();

                        countryItems.each(function (d, i) {
                            const item = d3.select(this);
                            const delay = currentIndex % 2 === 0 ?
                                (totalItems - i - 1) * 200 : // Reverse order for even indices (above the line)
                                i * 200; // Normal order for odd indices (below the line)

                            item.select("image")
                                .transition()
                                .delay(delay)
                                .duration(500)
                                .style("opacity", 1);

                            item.select("text")
                                .transition()
                                .delay(delay)
                                .duration(500)
                                .style("opacity", 1);
                        });

                        // Animate expansion line if it's an expansion year
                        const expansionLine = expansionLines.filter(d => d.year === eventData.year);
                        if (!expansionLine.empty()) {
                            expansionLine.transition()
                                .duration(500)
                                .style("opacity", 1);
                        }
                    }

                    // Start the animation
                    animateNextEvent();
                }
            }


        })();
    </script>
</body>

</html>
""")

In [9]:
display.HTML(viz1_html.safe_substitute({
    "df1": first_participation.to_csv(index=False),
    "df2": tournament_winners.to_csv(index=False),
    "flag_images": json.dumps(encoded_flags)
}))